In [51]:
import anthropic

# load dotenv for the api_key
import os
from dotenv import load_dotenv
load_dotenv()

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key = os.getenv("ANTHROPIC_API_KEY"),
)


import json

def save_data(message, name_of_file='augmented_dark_pattern_claude.json'):
    # Cargar los datos existentes
    try:
        with open(name_of_file, 'r') as f:
            resultados = json.load(f)
    except FileNotFoundError:
        print("File not found")
        print("Creating a new file")
        resultados = []

    # Agregar los nuevos datos
    nuevos_datos = json.loads(message.json())
    resultados.append(nuevos_datos)

    # Guardar los datos actualizados
    with open(name_of_file, 'w') as f:
        json.dump(resultados, f)

    print(f"Data saved in {name_of_file}")
    print("The length of the dataset after appending the new message is: ", len(resultados))

### Load original data

#### API Call to Augmente Data

In [53]:

message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0,
    system="Your role is to act as a data augmentation tool focused on generating data variations for dark patterns. Dark patterns are deceptive designs in user interfaces intended to make users behave in ways they do not intend.\n\nGiven the input data, generate three variations for data augmentation. Two variations should maintain the original label, while one should be altered significantly enough to belong to the opposite category. The categories for labeling are: 0 for 'Not Dark Pattern', and 1 for 'Dark Pattern'. Each variation should strive to maintain the core characteristics of the original input but alter aspects such as wording, structure, etc., to enhance model training diversity.\n\nAdditionally, for each augmented data, provide a 'Confidence' score between 0 and 1 that represents how well the new variations adhere to the assigned label and the quality of the variation.\n\nPlease return the output in the following JSON format:\n\n{\n  \"Original Data\": \"[Original Text]\",\n  \"Original Label\": \"[Appropriate Category]\",\n  \"Augmented Variations\": [\n    {\n      \"Augmented Data\": \"[Variation 1]\",\n      \"Label\": \"[Appropriate Category]\",\n      \"Confidence\": \"[Confidence Score]\"\n    },\n    {\n      \"Augmented Data\": \"[Variation 2]\",\n      \"Label\": \"[Appropriate Category]\",\n      \"Confidence\": \"[Confidence Score]\"\n    },\n    {\n      \"Augmented Data\": \"[Variation 3]\",\n      \"Label\": \"[Appropriate Category]\",\n      \"Confidence\": \"[Confidence Score]\"\n    }\n  ]\n}\n",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Original Data: \"To start your return, simply click on the following button and add your return details. Once the form is filled, you will receive an email with shipping instructions for how to ship the item(s) to our warehouse.\"\nOringinal Label: 0"
                }
            ]
        }
    ]
)
print(message.content)

# save_data(message)
name_of_file='augmented_dark_pattern_claude.json'
save_data(message, name_of_file)

[ContentBlock(text='{\n  "Original Data": "To start your return, simply click on the following button and add your return details. Once the form is filled, you will receive an email with shipping instructions for how to ship the item(s) to our warehouse.",\n  "Original Label": 0,\n  "Augmented Variations": [\n    {\n      "Augmented Data": "Click the button below to initiate your return. After completing the form, you\'ll get an email with instructions on how to send the item(s) to our warehouse.",\n      "Label": 0,\n      "Confidence": 0.9\n    },\n    {\n      "Augmented Data": "To begin the return process, press the button and provide the necessary return details. An email with shipping guidelines will be sent once the form is submitted.",\n      "Label": 0,\n      "Confidence": 0.8\n    },\n    {\n      "Augmented Data": "Attention shoppers! If you\'re not satisfied with your purchase, simply click the button to start the return process. But be warned, the return window is only op

### Load the JSON augmented file

In [58]:
# load the json file and print the content
with open(name_of_file, 'r') as f:
    augmented_data = json.load(f)

In [75]:
json.loads( augmented_data[0]["content"][0]["text"] )["Augmented Variations"][0]["Confidence"]

'0.95'

In [95]:
import numpy as np
results = {}
for i, msg in enumerate(augmented_data):
    model_used = msg["model"]
    content = msg['content'][0]
    conf_values = []
    for aug_var in json.loads(content["text"] )["Augmented Variations"]: 
        conf = aug_var["Confidence"]
        conf_values.append(conf)
    conf_values = np.array(conf_values).astype(float)
    print(f"confidence for message {i} using {model_used} -> {np.mean(conf_values)} ")
    results[i] = {"model_used": model_used, "confidence": conf_values }

confidence for message 0 using claude-3-opus-20240229 -> 0.9066666666666667 
confidence for message 1 using claude-3-haiku-20240307 -> 0.8000000000000002 


In [91]:
results

{0: {'model_used': 'claude-3-opus-20240229',
  'confidence': array([0.95, 0.92, 0.85])},
 1: {'model_used': 'claude-3-haiku-20240307',
  'confidence': array([0.9, 0.8, 0.7])}}